## Solutions - Problem 5

Get the total number of flights departed from the airports in January 2008 that do not contain entries in airport-codes.

In [1]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/JmQHuFdaOHo?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

* This is an example for outer join.
* We need to get number of flights from the 2008 January airtraffic data which do not have entries in airport-codes.
* Based on the side of the airtraffic data set, we can say left or right. We will be invoking join using airtraffic and hence we will use left outer join..
* We will be peforming join first and then we will aggregate to get number of flights from the concerned airports.
* In this case will get total number of flights.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Joining Data Sets'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [2]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [3]:
airtrafficPath = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [4]:
airtraffic = spark. \
    read. \
    parquet(airtrafficPath)

In [5]:
airtraffic. \
    select(
        "Year", "Month", "DayOfMonth", 
        "Origin", "Dest", "CRSDepTime"
    ). \
    show()

+----+-----+----------+------+----+----------+
|Year|Month|DayOfMonth|Origin|Dest|CRSDepTime|
+----+-----+----------+------+----+----------+
|2008|    1|        16|   BGR| CVG|      1735|
|2008|    1|        17|   SYR| CVG|      1701|
|2008|    1|        17|   SAV| BOS|      1225|
|2008|    1|        17|   CVG| GRR|      1530|
|2008|    1|        17|   STL| CVG|      1205|
|2008|    1|        18|   STL| JFK|      1150|
|2008|    1|        18|   MCI| CVG|      1009|
|2008|    1|        19|   TUL| CVG|       835|
|2008|    1|        20|   JFK| PHL|      1935|
|2008|    1|        20|   RDU| CVG|       830|
|2008|    1|        21|   CVG| DTW|      1640|
|2008|    1|        21|   MSY| LGA|      1204|
|2008|    1|        21|   JFK| PHL|      1935|
|2008|    1|        21|   DCA| JFK|      1830|
|2008|    1|        21|   HSV| DCA|       700|
|2008|    1|        22|   ORD| CVG|      1910|
|2008|    1|        22|   CVG| JFK|      1320|
|2008|    1|        23|   LGA| SAV|       908|
|2008|    1| 

In [6]:
airtraffic.count()

605659

In [7]:
airportCodesPath = "/public/airtraffic_all/airport-codes"

In [8]:
def getValidAirportCodes(airportCodesPath):
    airportCodes = spark. \
        read. \
        option("sep", "\t"). \
        option("header", True). \
        option("inferSchema", True). \
        csv(airportCodesPath). \
        filter("!(State = 'Hawaii' AND IATA = 'Big')")
    return airportCodes

In [9]:
airportCodes = getValidAirportCodes(airportCodesPath)

In [10]:
airportCodes.count()

524

In [11]:
airtraffic. \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA"], "left"). \
    filter("IATA IS NULL"). \
    select(airtraffic["Year"], airtraffic["Month"], airtraffic["DayOfMonth"], 
           airtraffic["Origin"], airtraffic["Dest"], airtraffic["CRSDepTime"], 
           airportCodes["*"]
          ). \
    show()

+----+-----+----------+------+----+----------+----+-----+-------+----+
|Year|Month|DayOfMonth|Origin|Dest|CRSDepTime|City|State|Country|IATA|
+----+-----+----------+------+----+----------+----+-----+-------+----+
|2008|    1|         8|   HDN| DEN|      1403|null| null|   null|null|
|2008|    1|        26|   HDN| DEN|      1533|null| null|   null|null|
|2008|    1|        26|   SJU| CLT|      1520|null| null|   null|null|
|2008|    1|        19|   SJU| ATL|       945|null| null|   null|null|
|2008|    1|        31|   ITO| HNL|      1010|null| null|   null|null|
|2008|    1|        26|   ITO| HNL|      1725|null| null|   null|null|
|2008|    1|        24|   KOA| LAX|      2355|null| null|   null|null|
|2008|    1|        22|   SJU| EWR|      1845|null| null|   null|null|
|2008|    1|        21|   SJU| MIA|      1815|null| null|   null|null|
|2008|    1|        26|   STT| MIA|       845|null| null|   null|null|
|2008|    1|        11|   SJU| MIA|      1235|null| null|   null|null|
|2008|

In [12]:
airtraffic. \
    join(airportCodes, airtraffic["Origin"] == airportCodes["IATA"], "left"). \
    filter("IATA IS NULL"). \
    count()

5585